In [1]:
import torch

In [2]:
"""
演示批量的BCELoss()。
网络的最后输出是sigmoid()函数，用于两分类问题。
"""
tensor1 = torch.Tensor([
    [0.4803],
    [0.9],
    [0.4906],
    [0.4804],
    [0.4925],
    [0.4789],
    [0.4760],
    [0.4803],
    [0.4786],
    [0.4727],
    [0.4827],
    [0.4727],
    [0.4789],
    [0.4749],
    [0.4778],
    [0.4811]])

tensor2 = torch.Tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor2_reshape = tensor2.view((16, 1))
loss_function = torch.nn.BCELoss()
loss = loss_function(tensor1, tensor2_reshape)
print(loss)

tensor(0.7568)


In [ ]:
# CrossEntropyLoss的用法
loss = nn.CrossEntropyLoss()
input = torch.randn(8, 2, requires_grad=True)
target = torch.empty(8, dtype=torch.long).random_(2)
print(input)
print(target)
output = loss(input, target)
print(output)

In [ ]:
z = torch.Tensor([-2.8961, -2.9042, -2.9938, -2.9543, -2.8207, -2.9718, -2.8264, -2.8576,
        -2.8961, -2.7781, -2.8565, -2.9186, -2.9661, -2.7642, -2.9291, -2.8690,
        -2.9487, -2.9125])
z = z.unsqueeze(0)
loss_func = nn.NLLLoss()
target = torch.Tensor([2]).long()
loss = loss_func(z, target)
print(loss)